In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/kalapa

/content/drive/My Drive/kalapa


In [ ]:
!pip install feature_engine
!pip install unidecode
!pip install category_encoders

     |████████████████████████████████| 9.5MB 4.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 81kB 2.3MB/s 


In [ ]:
from itertools import combinations

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings

import lightgbm as lgb
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from feature_engine import categorical_encoders as ce

from datetime import datetime
from contextlib import contextmanager

In [ ]:
lgbm_param = {'boosting_type': 'gbdt', 
              'metric': 'auc',
              #'colsample_bytree': 0.6602479798930369, 
              'is_unbalance': False, 
              'learning_rate': 0.01,
              'max_depth': -1, 
              'min_split_gain': 0.007, 
              'min_child_samples': 25, 
              'num_leaves': 50,
              'objective': 'binary', 
              'reg_alpha': 0.4693391197064131, 
              'reg_lambda': 0.16175478669541327, 
              'subsample_for_bin': 60000}

NUM_BOOST_ROUND= 10000

In [ ]:
DROP = (["gioiTinh","info_social_sex", 'currentLocationLocationId', 'currentLocationLatitude', 'currentLocationLongitude', 'homeTownLocationId', 'homeTownLatitude', 'homeTownLongitude', 'currentLocationName', 'homeTownName'] + 
[f"Field_{c}" for c in [11, 13, 14, 15, 16, 17, 18, 24, 25, 26, 30, 31, 32, 33, 34, 35, 37, 40, 45, 46, 48, 49, 52, 56, 57, 68]] + 
['partner0_B', 'partner0_K', 'partner0_L', 'partner1_B', 'partner1_D', 'partner1_E', 'partner1_F', 'partner1_K', 'partner1_L', 
   'partner2_B', 'partner2_G', 'partner2_K', 'partner2_L', 
   'partner3_B', 'partner3_C', 'partner3_F', 'partner3_G', 'partner3_H', 'partner3_K', 'partner3_L', 
   'partner4_A', 'partner4_B', 'partner4_C', 'partner4_D', 'partner4_E', 'partner4_F', 'partner4_G', 'partner4_H', 'partner4_K', 'partner4_L', 
   'partner5_B', 'partner5_C', 'partner5_D', 'partner5_H', 'partner5_K', 'partner5_L'])

cat_features_count_encode = ['Field_4', 'Field_12', 'Field_18', 'Field_34', 'gioiTinh', 'diaChi', 'Field_36',
                             'Field_38', 'Field_45', 'Field_46', 'Field_47', 'Field_48', 'Field_49',
                             'Field_54', 'Field_55', 'Field_56', 'Field_61', 'Field_62', 'Field_65', 'Field_66',
                             'Field_68', 'maCv', 'info_social_sex', 'data.basic_info.locale',
                             'currentLocationCountry', 'currentLocationState', 'homeTownCountry', 'homeTownState', 'brief']

unidecode_columns = 'diaChi currentLocationCountry currentLocationState homeTownCountry homeTownState maCv'.split()
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [ ]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

def lgb_gini(y_pred, dataset_true):
    y_true = dataset_true.get_label()
    return 'gini', Gini(y_true, y_pred), True

In [ ]:
def subtract_date(date1,date2, df):
    df[date1] = pd.to_datetime(df[date1], infer_datetime_format=True)
    df[date2] = pd.to_datetime(df[date2], infer_datetime_format=True)
    df[date1+date2] = (df[date2] - df[date1]).dt.days
    
def process_ngaySinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")

def datetime_normalize(s):
    if s != s:
        return np.nan
        
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj
  
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    df["Field_34"] = df["Field_34"].apply(process_ngaySinh)
    df["ngaySinh"] = df["ngaySinh"].apply(process_ngaySinh)
    
    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        #df[col].fillna(0)
        df[col] = df[col].dt.strftime('%m-%Y') #%d-%m-%Y
    
    subtract_date('Field_5','Field_6',df)
    subtrac_List = ['Field_1', 'Field_2', 'Field_43', 'Field_44', 'Field_7','Field_8', 'Field_9']
    subtract_2C = list(combinations(subtrac_List, 2))
    for l in subtract_2C:
        subtract_date(l[0],l[1],df)
      
    for cat in ['F', 'E', 'C', 'G', 'A']:
        subtract_date(f'{cat}_startDate', f'{cat}_endDate', df)
    for col in cat_cols:
        df[col] = df[col].astype("category")
    print(df.shape) 
    return df
  
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s

def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize).astype("category")
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize).astype("category")

    return df

def job_category(x):
    if type(x) == str:
        if "công nhân" in x or "cnv" in x or "cn" in x or "may công nghiệp" in x or "lao động" in x\
        or "thợ" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x or "c.n" in x or "lđ" in x:
            return "CN"
        elif "giáo viên" in x or "gv" in x or "gíao viên" in x:
            return "GV"
        elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x or "cb" in x or "nhõn viờn" in x:
            return "NV"
        elif "tài xế" in x or "lái" in x or "tài xê" in x:
            return "TX"
        elif "quản lý" in x or "phó phòng" in x or "hiệu phó" in x:
            return "QL"
        elif "undefined" in x:
            return "missing"
        elif "giám đốc" in x or "hiệu trưởng" in x:
            return "GĐ"
        elif "phục vụ" in x:
            return "PV"
        elif "chuyên viên" in x:
            return  "CV"
        elif "bác sĩ" in x or "dược sĩ" in x or "y sĩ" in x or "y sỹ" in x:
            return "BS"
        elif "y tá" in x:
            return "YT"
        elif "hộ sinh" in x:
            return "HS"
        elif "chủ tịch" in x:
            return "CT"
        elif "bếp" in x:
            return "ĐB"
        elif "sư" in x:
            return "KS"
        elif "dưỡng" in x:
            return "ĐD"
        elif "kỹ thuật" in x or "kĩ thuật" in x:
            return "KTV"
        elif "diễn viên" in x:
            return "DV"
        else:
            return "missing"
    else:
        return x 

def process_diaChi_maCv(df):
    df["maCv"] = df["maCv"].apply(str_normalize).apply(job_category).astype("category")
    return df

def combine_gender(s):
    x, y = s
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
    return df

def process_ordinal(df):
    print(df.columns)

    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)

    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1.0, "II": 2.0, "III": 3.0, "IV": 4.0, "V": 5.0, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0.0, "One": 1.0, "Two": 2.0, "Three": 3.0, "Four": 4.0})
    df["Field_66"] = df["Field_66"].map({"B": 0.0, "C": 1.0, "D": 2.0, "E": 3.0, "F": 4.0, "G": 5.0, "H": 6.0, "I": 7.0})

    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df

def transform(df):
    df["gioiTinh"] = df["gioiTinh"].str.lower()
    df["maCv"] = df["maCv"].str.lower()
    df = process_datetime_cols(df)
    df = process_gender(df)
    df = process_location(df)
    df = process_diaChi_maCv(df)
    df = process_ordinal(df)
    #df[unidecode_columns] = df[unidecode_columns].applymap(lambda x: unidecode(x).lower() if x == x else x)
    #print(df[unidecode_columns])
    #df[cat_features_count_encode] = df[cat_features_count_encode].astype('category')
    df["null_sum"] = df.isnull().sum(axis=1)
    return df.drop(DROP, 1)

In [ ]:
def kfold(train_fe,y_label,test_fe):
    seeds = np.random.randint(0, 10000, 1)
    preds = 0    
    feature_important = None
    avg_train_gini = 0
    avg_val_gini = 0

    for s in seeds:
        skf = StratifiedKFold(n_splits=5, random_state = 6484, shuffle=True)        
   
        seed_train_gini = 0
        seed_val_gini = 0
        for i, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y_label)), y_label)):                
            X_train, X_val = train_fe.iloc[train_idx].drop(["id"], 1), train_fe.iloc[val_idx].drop(["id"], 1)                
            y_train, y_val = y_label[train_idx], y_label[val_idx]

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval  = lgb.Dataset(X_val, y_val)
          
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
            evals_result = {}
            model = lgb.train(lgbm_param,
                        lgb_train,
                        num_boost_round=NUM_BOOST_ROUND,  
                        early_stopping_rounds=800,
                        feval=lgb_gini,
                        verbose_eval= 200,
                        evals_result=evals_result,
                        valid_sets=[lgb_train, lgb_eval])

            seed_train_gini += model.best_score["training"]["gini"] / skf.n_splits
            seed_val_gini += model.best_score["valid_1"]["gini"] / skf.n_splits

            avg_train_gini += model.best_score["training"]["gini"] / (len(seeds) * skf.n_splits)
            avg_val_gini += model.best_score["valid_1"]["gini"] / (len(seeds) * skf.n_splits)

            if feature_important is None:
                feature_important = model.feature_importance() / (len(seeds) * skf.n_splits)
            else:
                feature_important += model.feature_importance() / (len(seeds) * skf.n_splits)        

            pred = model.predict(test_fe.drop(["id"], 1))
            preds += pred / (skf.n_splits * len(seeds))

            print("Fold {}: {}/{}".format(i, model.best_score["training"]["gini"], model.best_score["valid_1"]["gini"]))  
        print("Seed {}: {}/{}".format(s, seed_train_gini, seed_val_gini))

    print("-" * 50)
    print("Avg train gini: {}".format(avg_train_gini))
    print("Avg valid gini: {}".format(avg_val_gini))
    print("=" * 50)
    return preds

In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/kalapa/dataset/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/kalapa/dataset/test.csv')
df_all = df_train.drop(['label'], 1).append(df_test)
df_train.fillna(-1, inplace=True)
df_test.fillna(-1, inplace=True)

df_all.replace("None", -1, inplace=True)
df_all.replace("Missing", -999, inplace=True)
df_all["gioiTinh"] = df_train["gioiTinh"].str.lower()

with timer("Preprocess"):
    df_all_fe = transform(df_all.copy())
    print("Bureau df shape:", df_all_fe.shape)
    df_all_fe['Age'] = df_all_fe.ngaySinh.apply(lambda x: 2020 - x.year)
    df_all_fe = df_all_fe.drop('ngaySinh', axis = 1)
    cols_select = [x for x in df_all_fe.columns if x not in DATE + DATETIME  + [f'{cat}_endDate' for cat in ['F', 'E', 'C', 'G', 'A']] + [f'{cat}_startDate' for cat in ['F', 'E', 'C', 'G', 'A']]]
    df_fe = df_all_fe[cols_select]
    df_fe.replace([np.inf, -np.inf], -99999, inplace=True)

    y_label = df_train["label"]
    train_fe = df_fe[df_fe["id"] < df_train.shape[0]]
    test_fe = df_fe[df_fe["id"] >= df_train.shape[0]]

    # Label-Encoding
    lbl = LabelEncoder()
    for col in df_fe.columns:
      if df_fe[col].dtype.name == "category":
        train_fe[col] = train_fe[col].astype(str)
        test_fe[col] = test_fe[col].astype(str)
        encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
        train_fe[col] = train_fe[col].fillna('None')
        test_fe[col] = test_fe[col].fillna('None')

        # Only take the common values in Train/Test set
        common_vals = list(set(train_fe[col]).intersection(set(test_fe[col])))

        # Take if vals appeared both 5 times
        common_vals = set(train_fe[col].value_counts()[train_fe[col].value_counts() > 4].index).intersection(test_fe[col].value_counts()[test_fe[col].value_counts()>4].index)

        # Replace not-common values with "Missing" or np.nan
        train_fe.loc[~train_fe[col].isin(common_vals), col] = 'Missing'
        test_fe.loc[~test_fe[col].isin(common_vals), col] = 'Missing'

        # Implement LE
        lbl.fit(train_fe[col].tolist() + test_fe[col].tolist())
        train_fe[col] = lbl.transform(train_fe[col])
        test_fe[col] = lbl.transform(test_fe[col])

    print(train_fe.shape)
    print(test_fe.shape) 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(73411, 221)
Index(['id', 'Field_1', 'Field_2', 'Field_3', 'Field_4', 'Field_5', 'Field_6',
       'Field_7', 'Field_8', 'Field_9',
       ...
       'Field_44Field_9', 'Field_7Field_8', 'Field_7Field_9', 'Field_8Field_9',
       'F_startDateF_endDate', 'E_startDateE_endDate', 'C_startDateC_endDate',
       'G_startDateG_endDate', 'A_startDateA_endDate', 'gender'],
      dtype='object', length=222)
Bureau df shape: (73411, 151)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

(53030, 132)
(20381, 132)
Preprocess - done in 17s


In [ ]:
with timer("Kfold"):
    preds = kfold(train_fe,y_label,test_fe)
    df_test["label"] = preds
    df_test[['id', 'label']].to_csv('submission.csv', index=False)
#display_importances(feature_importance_df)   

Training until validation scores don't improve for 800 rounds.
[200]	training's auc: 0.770316	training's gini: 0.540632	valid_1's auc: 0.733046	valid_1's gini: 0.466089
[400]	training's auc: 0.799762	training's gini: 0.599525	valid_1's auc: 0.740892	valid_1's gini: 0.481781
[600]	training's auc: 0.823568	training's gini: 0.647135	valid_1's auc: 0.74515	valid_1's gini: 0.4903
[800]	training's auc: 0.842453	training's gini: 0.684907	valid_1's auc: 0.747033	valid_1's gini: 0.494061
[1000]	training's auc: 0.857797	training's gini: 0.715592	valid_1's auc: 0.747736	valid_1's gini: 0.495473
[1200]	training's auc: 0.870806	training's gini: 0.741612	valid_1's auc: 0.748326	valid_1's gini: 0.496646
[1400]	training's auc: 0.881676	training's gini: 0.76335	valid_1's auc: 0.748658	valid_1's gini: 0.497318
[1600]	training's auc: 0.891555	training's gini: 0.783108	valid_1's auc: 0.749086	valid_1's gini: 0.49817
[1800]	training's auc: 0.900332	training's gini: 0.800665	valid_1's auc: 0.749352	valid_1'

0.5082305588525496